In [1]:
import json
import urllib.request
from unidecode import unidecode

def request(action, **params):
    return {'action': action, 'params': params, 'version': 6}

def invoke(action, **params):
    requestJson = json.dumps(request(action, **params)).encode('utf-8')
    response = json.load(urllib.request.urlopen(urllib.request.Request('http://127.0.0.1:8765', requestJson)))
    if len(response) != 2:
        raise Exception('response has an unexpected number of fields')
    if 'error' not in response:
        raise Exception('response is missing required error field')
    if 'result' not in response:
        raise Exception('response is missing required result field')
    if response['error'] is not None:
        raise Exception(response['error'])
    return response['result']

invoke('createDeck', deck='test1')
result = invoke('deckNames')
print('got list of decks: {}'.format(result))

got list of decks: ['-An Verbs in Tagalog', 'Adjectives in Tagalog', 'Default', 'test1']


In [36]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [37]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import requests
from bs4 import BeautifulSoup

In [39]:
#driver = webdriver.Chrome(ChromeDriverManager().install())

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:
from IPython.display import HTML

In [5]:
import re

regex = re.compile('[^a-zA-Z]')

In [6]:
result

['-An Verbs in Tagalog', 'Adjectives in Tagalog', 'Default', 'test1']

In [7]:
find_cards_params = {
    "query": "deck:current"
}

In [8]:
result = invoke('findCards', **find_cards_params)
print('got list of card IDs: {}'.format(result))

got list of card IDs: [1718165006067, 1718165399899, 1718165403407, 1718165448684, 1718165451131, 1718165453904, 1718165456450, 1718165459000, 1718165463000, 1718165465879, 1718165468749, 1718165471201, 1718165473598, 1718165476152, 1718165478677, 1718165834427, 1718165837393, 1718165840447, 1718165842893, 1718166264873, 1718166267372, 1718166270247, 1718166272974, 1718166486163, 1718166500919, 1718166503518, 1718166506643, 1718166534070, 1718166851349, 1718166854219, 1718166856891, 1718166859543, 1718166862788, 1718166866274, 1718166868919, 1718166871493, 1718166874108, 1718166876835, 1718166879482, 1718166882189, 1718166884939, 1718166887889, 1718166890445, 1718166892939, 1718166895568, 1718166898165, 1718166900739, 1718166903365, 1718166906008, 1718166908691, 1718166911294, 1718166913788, 1718166916332, 1718166918919, 1718166921484, 1718166923985, 1718167123655, 1718167126358, 1718167128838, 1718167739338, 1718167742007, 1718167744650, 1718167784456, 1718167787474, 1718167790127, 17

In [9]:
cards_info_params = {
    "cards": result
}
cards_info_result = invoke('cardsInfo', **cards_info_params)

In [10]:
#cards_info_result[10]['fields']['Front']['value'].split('<br>', 1)[0]
unidecode(re.sub(r'<[^>]*>', '', cards_info_result[2]['fields']['Front']['value'].split('<br>', 1)[0]).strip())

'pangit'

In [11]:
def check_if_word_exists(input_word):
    find_cards_params = {
    "query": "deck:current"
    }
    result = invoke('findCards', **find_cards_params)
    cards_info_params = {
    "cards": result
    }
    cards_info_result = invoke('cardsInfo', **cards_info_params)
    for c in range(0, len(result)):
        curr_word = unidecode(re.sub(r'<[^>]*>', '', cards_info_result[c]['fields']['Front']['value'].split('<br>', 1)[0]).strip())
        if (input_word == curr_word.strip()) or (input_word == curr_word.replace('-', '').strip()) or (input_word.replace('-', ' ').strip() == curr_word):
            return True
    return False


In [12]:
check_if_word_exists('walisan')

False

In [14]:
new_card_data = {
    'Front': '<span style="color:#7EC8E3; font-weight:bold; font-size:18px;">[verb]</span>  to allow something; to let something be; to tolerate something; to let someone/something do something; to leave something; to not disturb something; to let something happen; to never mind something',
    'Back': 'to skin something; to peel something; to pare something (remove the skin); to de-skin something'
}



add_note_params = {
    'note': {
        'deckName': 'test1', 
        'modelName': 'Basic',
        'fields': new_card_data
            }
    }



invoke('addNote', **add_note_params)

Exception: cannot create note because it is a duplicate

# Beautiful Soup Parsing Section

# NEXT STEPS:
1. create function based on above block to find the correct word link based on matching part of speech. A simple search through the parent div for [verb], [noun], [adjective] etc. will do. Return the link to the corresponding page
2. Then Create the function which pull relevant data from word-specific page and returns it
3. Then create function which adds new card to deck based on previous function
   

In [15]:
def retrieve_word_html(url):
    thisurl = url
    page = requests.get(thisurl)
    soup = BeautifulSoup(page.text, 'html')
    response = requests.get(url)
    html_content = response.text

    
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup

In [16]:
def mine_word(page_html):
    soup = page_html
    html_string_word = str(soup.find("b", {"id": "wordbox"}))
    print(html_string_word)
    split_result_word = html_string_word.split(';">')
    
    
    desired_text_word = split_result_word[1]
    
    
    end_index_word = desired_text_word.find('</b>')
    
    
    extracted_text_word = desired_text_word[:end_index_word]
    
    extracted_text_word = extracted_text_word.strip()
    return extracted_text_word

In [17]:
def part_of_speech_function(page_html):
    #CHECK IF WORD IS VERB, if so, initialize html for [verb] element
    soup = page_html
    type_of_speech = regex.sub('', soup.find("div", {"id": "definitionbox"}).find('span').get_text(strip=True))
    if type_of_speech == '':
        type_of_speech = regex.sub('', soup.find("div", {"id": "definitionbox"}).find_all('span')[1].get_text(strip=True))
    if  type_of_speech == 'verb':
        part_of_speech = '<span style="color:#7EC8E3; font-weight:bold; font-size:22px;">[verb]</span>'
    elif type_of_speech == 'noun':
        part_of_speech = '<span style="color:#7EC8E3; font-weight:bold; font-size:22px;">[noun]</span>'
    elif type_of_speech == 'adjective':
        part_of_speech = '<span style="color:#7EC8E3; font-weight:bold; font-size:22px;">[adjective]</span>'
    else:
        return type_of_speech, None
    return type_of_speech, part_of_speech
    

In [18]:
def extract_tenses_and_conjugations(page_html):
    soup = page_html
    conjugation_table = soup.find("div", {"class":"bigscreenonly"})
    
    for audio in conjugation_table.find_all('audio'):
        audio.extract()
    for img in conjugation_table.find_all('img'):
        img.extract()
    for a in conjugation_table.find_all('a'):
        del a['href']
    
    soup = conjugation_table
    
    headers = soup.find_all(class_='aspect-table-header')
    #"""<span style="color:#f2f2dd; font-weight:bold; font-size:22px;">""" + Conjugation Type + """</span>"""
    tenses = ["""<span style="color:#f2f2dd; font-weight:bold; font-size:22px;">""" + header.get_text(strip=True) + """</span>""" for header in headers]
    
    
    entries = soup.find_all(class_='aspect-table-entry')
    conjugations = [entry.find('a').prettify() for entry in entries]
    
    return tenses, conjugations

In [19]:
#a,b = extract_tenses_and_conjugations(retrieve_word_html(url))

In [20]:
def extract_focus(page_html):
    soup = page_html
    #extracted_text_focus = soup.find('span', class_='conjugation-detail-content').get_text(strip=True)
    try:
        extracted_text_focus = soup.find('span', class_='conjugation-detail').find('span', class_='conjugation-detail-content').get_text(strip=True)
    except:
        print('no focus found!')
        return None
    if '-An' == str(extracted_text_focus):
        return None
    return extracted_text_focus

In [21]:
def extract_root(page_html):
    soup = page_html
    extracted_text_root = soup.find('a', class_='conjugation-detail').find('span', class_='conjugation-detail-content').get_text(strip=True)
    return extracted_text_root

In [22]:
extracted_text_focus = soup.find('span', class_='conjugation-detail-content').get_text(strip=True)
extracted_text_root = soup.find('a', class_='conjugation-detail').find('span', class_='conjugation-detail-content').get_text(strip=True)

NameError: name 'soup' is not defined

In [23]:
#if type_of_speech == 'verb':
#    extracted_text_conjugationtype = soup.find_all('span', class_='conjugation-detail')[1].find('span', class_='conjugation-detail-content').get_text(strip=True)
#else:
#    extracted_text_conjugationtype = None

In [24]:
# TABLE VERSION BROTHA re.sub(r'(?<!<)\s+(?![^<>]*>)', '', b[0].replace('\n', '').replace('15px', '20px'))
def create_back_of_card(conjugation_list, definition):
    extracted_text_definition = definition
    b = conjugation_list
    base_string_one = """<table>
  <tr>
    <th style="text-align: center";font-size:40px;>Infinitive</th>
    <th style="text-align: center">Completed (Past)</th>
    <th style="text-align: center">Uncompleted (Present)</th>
    <th style="text-align: center">Contemplated (Future)</th>
  </tr>
  <tr>
    <td style="text-align: center">{0}</td>
    <td style="text-align: center">{1}</td>
    <td style="text-align: center">{2}</td>
    <td style="text-align: center">{3}</td>
  </tr>
</table>""".format(re.sub(r'(?<!<)\s+(?![^<>]*>)', '', b[0].replace('\n', '').replace('15px', '16px')), re.sub(r'(?<!<)\s+(?![^<>]*>)', '', b[1].replace('\n', '').replace('15px', '16px')), re.sub(r'(?<!<)\s+(?![^<>]*>)', '', b[2].replace('\n', '').replace('15px', '16px')), re.sub(r'(?<!<)\s+(?![^<>]*>)', '', b[3].replace('\n', '').replace('15px', '16px')))
    base_string_two = """{}<br><br>{}""".format(extracted_text_definition, base_string_one)
    return base_string_two

In [25]:
#create_back_of_card(b)

In [26]:
def create_front_and_back(url, input_word):
    page_html_input = retrieve_word_html(url)
    extracted_text_word = mine_word(page_html_input)
    type_of_speech, part_of_speech = part_of_speech_function(page_html_input)
    extracted_text_definition = page_html_input.find("div", {"id": "definitionbox"}).contents[0]#.decode_contents().replace(' color:#555',' color:#7EC8E3')
    
    for img in extracted_text_definition.find_all('img'):
        img.extract()

    extracted_text_definition = extracted_text_definition.decode_contents().replace(' color:#555',' color:#7EC8E3').replace('rgba(255,255,255,0.2)',  'rgba(60,150,0,0.4)').replace('#333', '#3C9600')
    
    if type_of_speech == 'verb':
        a,b = extract_tenses_and_conjugations(page_html_input)
        extracted_text_focus = extract_focus(page_html_input)
    extracted_text_root = extract_root(page_html_input)
    
    #print(extracted_text_focus)
    #print(extracted_text_root)
    
    if type_of_speech == 'verb':
        if extracted_text_focus is None:
            try:
                extracted_text_conjugationtype = page_html_input.find('span', class_='conjugation-detail').find('span', class_='conjugation-detail-content').get_text(strip=True)
                if extracted_text_conjugationtype == '-An':
                    extracted_text_focus = 'Locative/Benefactive'
            except:
                # do a check to see if the last two letters are -an
                print('went into the weird branch and there is no focus or conjugation type')
                if input_word.endswith('an'):
                    extracted_text_conjugationtype = '-An'
                elif (input_word.startswith('ma') and input_word.endswith('an')):
                    extracted_text_conjugationtype = 'Ma- -An'
                else:
                    print('and i TRIED to fit something in here but its not happening...')
                    extracted_text_conjugationtype = ''
        else:
            try:
                extracted_text_conjugationtype = page_html_input.find_all('span', class_='conjugation-detail')[1].find('span', class_='conjugation-detail-content').get_text(strip=True)
            except:
                print('no conjugation detail found!')
                extracted_text_conjugationtype = ''
    if type_of_speech == 'verb':
        back_of_the_card = create_back_of_card(b, extracted_text_definition)
    #base_string_one = '{}<br> <br> <span style="color:#f2f2dd; font-weight:bold; font-size:22px;">Focus:</span> {}'.format(extracted_text_word, extracted_text_focus)
    
    if type_of_speech == 'verb':
        base_string_one = '{}<br> <br> <span style="color:#f2f2dd; font-weight:bold; font-size:22px;">Focus:</span> {}'.format(extracted_text_word, extracted_text_focus)
        base_string_two = '{}<br> <span style="color:#f2f2dd; font-weight:bold; font-size:22px;">Root:</span> {}'.format(base_string_one, extracted_text_root)
        base_string_three = '{}<br> <span style="color:#f2f2dd; font-weight:bold; font-size:22px;">Conjugation Type:</span> {}'.format(base_string_two,extracted_text_conjugationtype)
        front_of_card = base_string_three
        #final_back_of_card = part_of_speech + ' ' + back_of_the_card
        final_back_of_card = back_of_the_card
    else:
        base_string_two = '{}<br> <br> <span style="color:#f2f2dd; font-weight:bold; font-size:22px;">Root:</span> {}'.format(extracted_text_word, extracted_text_root)
        front_of_card = base_string_two
        #final_back_of_card = part_of_speech + ' ' + extracted_text_definition
        final_back_of_card = extracted_text_definition
    return front_of_card, final_back_of_card
        
    
    
    

In [27]:
url = 'https://www.tagalog.com/dictionary/walisan'
front, back = create_front_and_back(url)
new_card_data = {
        'Front':  front,
        'Back': back
}



add_note_params = {
    'note': {
        'deckName': 'test1',  # Deck where the new card will be added
        'modelName': 'Basic',  # Note type
        'fields': new_card_data
            }
    }



invoke('addNote', **add_note_params)

TypeError: create_front_and_back() missing 1 required positional argument: 'input_word'

In [50]:
def make_selenium_connection(word_of_interest):
    chromedriver_path = '/Users/rubenverghese/bin/chromedriver'

    
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Run Chrome in headless mode
    # OLD VERSION?
    #service = Service(chromedriver_path)
    #driver = webdriver.Chrome(service=service, options=chrome_options)

    
    service = Service(ChromeDriverManager().install())
    
    
    driver = webdriver.Chrome(service=service, options=chrome_options)

    temp_word = '#' + str(word_of_interest)
    
    url = 'https://www.tagalog.com/dictionary/' + temp_word
    
    
    driver.get(url)
    
    
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, 'search_result1'))
    )
    
    html_content = driver.page_source
    
    html_after_loaded = BeautifulSoup(html_content, 'html.parser')
    
    # Print the found element
    #print("Element found:", element.get_attribute('outerHTML'))

    
    
    
    driver.quit()

    
    return html_after_loaded

In [29]:
def extract_word_from_url(url):
    
    pattern = r"https://www.tagalog.com/dictionary/(.*?)-\d+"
    
   
    match = re.search(pattern, url)
    
    
    if match:
        word = match.group(1)
        return word
    else:
        return url.split("/")[-1]

In [30]:
def find_correct_word(word, force_part_of_speech = None):
    html_of_word = make_selenium_connection(word)

    counter_to_return = None
    
    search_element_exists = True
    search_counter=1
    while search_element_exists == True:
        if html_of_word.find("a", {"id":"search_result" + str(search_counter)}) is None:
            search_element_exists = False
            print('no_search_element_exists')
            counter_to_return = search_counter
            break
        else:
            sub_html = html_of_word.find("a", {"id":"search_result" + str(search_counter)})
            print(force_part_of_speech)
            if force_part_of_speech is not None:
                print('made_it_here')
                input_part_of_speech = '[' + str(force_part_of_speech.strip().lower()) + ']'
                print(str(sub_html.find_parent("div")).find(input_part_of_speech))
                if str(sub_html.find_parent("div")).find(input_part_of_speech) == -1:
                    search_counter = search_counter + 1
                    continue
            newlink = sub_html['href']
            curr_word = extract_word_from_url(newlink)
            # Made this modification in this version
            if (word == curr_word.replace('-', '').strip()) or (word == curr_word):
                return newlink
            else:
                print('search_element_' + str(search_counter) + ' exists,' + 'and it is: ' + str(curr_word))
        search_counter = search_counter + 1
        
    return counter_to_return

In [31]:
words_to_add = ['lakaran','talikuran','hanapan','hugasan','asahan','takbuhan','salihan','sundan','samahan','sigawan','tawanan','basahan','lapitan','turuan','dalhan','gawan','bawasan','bigyan','tulungan','sulatan','pakinggan','puntahan','buksan', 'lakasan', 'hayaan','walisan', 'tingnan', 'pag-usapan', 'pag-isipan', 'bilhan', 'maramdaman', 'damihan']
words_to_add += ['mamalayan']

In [51]:
for c in words_to_add:
    if check_if_word_exists(c):
        print(str(c) + ' is already in the deck!')
        continue
    url = find_correct_word(c, 'verb')
    front, back = create_front_and_back(url, c)
    new_card_data = {
            'Front':  front,
            'Back': back
    }
    
    
    # Create a new note with the specified fields
    add_note_params = {
        'note': {
            'deckName': '-An Verbs in Tagalog',  # Deck where the new card will be added
            'modelName': 'Basic',  # Note type
            'fields': new_card_data
                }
        }
    
    
    # Add the new note to the deck
    invoke('addNote', **add_note_params)

verb
made_it_here
1163
<b id="wordbox" style="display:inline-block; margin-top:2px; letter-spacing:1px;">lak<span style="  border-bottom:2px solid !important;  font-size:inherit; ">a</span>ran</b>


Exception: cannot create note because it is a duplicate

In [33]:
c = 'bilhan'
url = find_correct_word(c, 'verb')
front, back = create_front_and_back(url, c)
new_card_data = {
        'Front':  front,
        'Back': back
}



add_note_params = {
    'note': {
        'deckName': '-An Verbs in Tagalog',  # Deck where the new card will be added
        'modelName': 'Basic',  # Note type
        'fields': new_card_data
            }
    }



invoke('addNote', **add_note_params)

SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 124
Current browser version is 129.0.6668.101 with binary path /Applications/Google Chrome.app/Contents/MacOS/Google Chrome
Stacktrace:
0   chromedriver                        0x00000001024f28dc chromedriver + 4368604
1   chromedriver                        0x00000001024ead70 chromedriver + 4337008
2   chromedriver                        0x000000010210ec04 chromedriver + 289796
3   chromedriver                        0x0000000102142b7c chromedriver + 502652
4   chromedriver                        0x0000000102141e5c chromedriver + 499292
5   chromedriver                        0x000000010213dd30 chromedriver + 482608
6   chromedriver                        0x000000010213a704 chromedriver + 468740
7   chromedriver                        0x00000001021791e0 chromedriver + 725472
8   chromedriver                        0x0000000102145ab4 chromedriver + 514740
9   chromedriver 

In [177]:
word_to_add_now = 'lapatan'
if check_if_word_exists(word_to_add_now):
        print(str(word_to_add_now) + ' is already in the deck!')
else:
    url = find_correct_word(word_to_add_now, 'verb')
    front, back = create_front_and_back(url, word_to_add_now)
    new_card_data = {
            'Front':  front,
            'Back': back
    }
    
    
    
    add_note_params = {
        'note': {
            'deckName': '-An Verbs in Tagalog',  
            'modelName': 'Basic',  
            'fields': new_card_data
                }
        }
    
    
    
    invoke('addNote', **add_note_params)

verb
made_it_here
1163
<b id="wordbox" style="display:inline-block; margin-top:2px; letter-spacing:1px;">lap<span style="  border-bottom:2px solid !important;  font-size:inherit; ">a</span>tan</b>


In [122]:
words_to_add = ['kulang', 'pangit', 'matangkad', 'pandak', 'malaki', 'maliit', 'mahaba', 'maikli', 'mataba', 'payat']
words_to_add += ['malambot', 'matigas', 'marunong', 'mahilig', 'mahal', 'mura', 'kulang', 'matalik', 'matarik']
words_to_add += ['maalat', 'matamis', 'mapait', 'maasim', 'madulas', 'malangis', 'makinis', 'magaspang', 'walang-laman']
words_to_add += ['puno', 'mali', 'tunay', 'tama', 'totoo', 'peke', 'huwad', 'malinis', 'marumi', 'matalino', 'masipag']
words_to_add += ['tamad', 'malapit', 'malayo', 'galit', 'masaya', 'malungkot', 'makulit', 'maayos', 'mahina', 'malakas']
words_to_add += ['mabaho', 'mabango', 'maingay', 'malalim', 'mababaw', 'maliwanag', 'makulimlim', 'malamig', 'mainit']
words_to_add += ['matulungin', 'nakakainis', 'malutong', 'makunat', 'nakakayamot', 'nakatira', 'makulay', 'ningning']
words_to_add += ['matalas', 'makati', 'matilos']
words_to_add += ['bago', 'luma', 'mataas', 'mababa', 'mabagal', 'maaga', 'huli', 'mabigat', 'magaan', 'basa']
words_to_add += ['masama','tuyo', 'masikip', 'maluwang', 'makapal', 'manipis', 'tapat', 'mapagbigay', 'kuripot', 'matakaw', 'mabait']
words_to_add += ['magaling']
words_to_add += ['matapang', 'malusog', 'duwag', 'sariwa', 'bulok', 'magulo', 'nakakatakot', 'nakakatawa', 'siksik']
words_to_add += ['manipis','makapal', 'mabuti', 'pagod', 'mabilis', 'marami']
words_to_add += ['madaldal', 'mahirap', 'madali', 'mabilis', 'maikli']
words_to_add += ['mapagkakatiwalaan', 'mapagmahal', 'mapayapa', 'mahinahon', 'maulan', 'mahangin']
words_to_add += ['mabuhangin', 'malabo', 'bagsak', 'matulis', 'mabagsik', 'malakas', 'mahinahon']
words_to_add += ['malamlam', 'makintab', 'matibay', 'maluho', 'maluwag', 'magara', 'mabenta', 'malupit']
words_to_add += ['matapat', 'masaya', 'maasikaso', 'maluwalhati', 'malambing', 'mapanganib', 'masakit', 'matuwa']


In [123]:
for c in words_to_add:
    if check_if_word_exists(c):
        print(str(c) + ' is already in the deck!')
        continue
    url = find_correct_word(c)
    front, back = create_front_and_back(url)
    new_card_data = {
            'Front':  front,
            'Back': back
    }
    
    
    
    add_note_params = {
        'note': {
            'deckName': 'Adjectives in Tagalog',  
            'modelName': 'Basic',  
            'fields': new_card_data
                }
        }
    
    
    
    invoke('addNote', **add_note_params)

kulang is already in the deck!
pangit is already in the deck!
matangkad is already in the deck!
pandak is already in the deck!
malaki is already in the deck!
maliit is already in the deck!
mahaba is already in the deck!
maikli is already in the deck!
mataba is already in the deck!
payat is already in the deck!
malambot is already in the deck!
matigas is already in the deck!
marunong is already in the deck!
mahilig is already in the deck!
mahal is already in the deck!
mura is already in the deck!
kulang is already in the deck!
matalik is already in the deck!
matarik is already in the deck!
maalat is already in the deck!
matamis is already in the deck!
mapait is already in the deck!
maasim is already in the deck!
madulas is already in the deck!
malangis is already in the deck!
makinis is already in the deck!
magaspang is already in the deck!
walang-laman is already in the deck!
puno is already in the deck!
mali is already in the deck!
tunay is already in the deck!
tama is already in the 

In [91]:
c = 'nakakinis'
if check_if_word_exists(c):
    print(str(c) + ' is already in the deck!')
else:
    url = find_correct_word(c, 'adjective')
    front, back = create_front_and_back(url)
    new_card_data = {
            'Front':  front,
            'Back': back
    }
    
    
    
    add_note_params = {
        'note': {
            'deckName': 'Adjectives in Tagalog',  
            'modelName': 'Basic',  
            'fields': new_card_data
                }
        }
    
    
    
    invoke('addNote', **add_note_params)

adjective
made_it_here
-1
adjective
made_it_here
-1
adjective
made_it_here
1336
search_element_3 exists,and it is: makinis
adjective
made_it_here
-1
no_search_element_exists


MissingSchema: Invalid URL '5': No scheme supplied. Perhaps you meant https://5?